In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date, timedelta
import time
import os




In [7]:
old_data = pd.read_csv('../dailyMontrealEdStats.csv')
old_data['date'] = pd.to_datetime(old_data['date'])
print('old data: ', len(old_data), ' rows, ending ', old_data.date.max())
old_data.head()


old data:  6293  rows, ending  2021-03-08 00:00:00


,Installation,Taux occupation civières(%),Nombre de civières fonctionnelles,Nombre de patients sur civière,Nombre de patients sur civière +24h,Nombre de patients sur civière +48h,Nombre patients arrivés en ambulance,Nombre inscriptions,Nombre inscriptions avec SAG,date
0,CHU Sainte-Justine,94,16,15,0,0,16,274,90,2020-01-25
1,Centre hospitalier de l'Université de Montréal,145,51,74,36,13,48,174,0,2020-01-25
2,Hôpital Louis-H. Lafontaine,200,12,24,20,15,1,9,0,2020-01-25
3,Hôpital Santa Cabrini,97,38,37,12,1,38,103,0,2020-01-25
4,Pavillon Maisonneuve/Pavillon Marcel-Lamoureux,115,54,62,18,2,57,194,6,2020-01-25


In [8]:
url = 'https://santemontreal.qc.ca/fileadmin/fichiers_portail/Donnees_urgence/urgence_quotidien_media.html'
r = requests.get(url)
soup = BeautifulSoup(r.text)

table = soup.find('table')

rows = table.find_all('tr')
table_rows = rows[0].find_all('tr')

columns = [i.text.replace('-', '') for i in table_rows[6].find_all('td')]

df_rows = []
for tr in table_rows[7:][:-6]:
    td = tr.find_all('td')
    row = [i.text for i in td]
    if '\xa0' in str(tr):
        continue
    if 'Sous-total' in str(tr):
        continue
    df_rows.append(row)

df = pd.DataFrame.from_records(df_rows, columns=columns)

df.head()


,Installation,Taux occupation civières(%),Nombre de civières fonctionnelles,Nombre de patients sur civière,Nombre de patients sur civière +24h,Nombre de patients sur civière +48h,Nombre patients arrivés en ambulance,Nombre inscriptions,Nombre inscriptions avec SAG
0,CHU Sainte-Justine,31,16,5,0,0,7,186,2
1,Centre hospitalier de l'Université de Montréal,114,51,58,21,7,48,165,0
2,Hôpital Louis-H. Lafontaine,42,12,5,3,1,0,13,0
3,Hôpital Santa Cabrini,129,38,49,17,11,36,91,0
4,Pavillon Maisonneuve/Pavillon Marcel-Lamoureux,131,54,71,21,4,41,152,0


In [9]:
columns.remove('Installation')

for int_col in columns:
    df[int_col] = pd.to_numeric(df[int_col], errors='coerce')

yesterday = date.today() - timedelta(days=1)
df['date'] = yesterday
df.date = pd.to_datetime(df.date)

new_data = df

new_data.head()

,Installation,Taux occupation civières(%),Nombre de civières fonctionnelles,Nombre de patients sur civière,Nombre de patients sur civière +24h,Nombre de patients sur civière +48h,Nombre patients arrivés en ambulance,Nombre inscriptions,Nombre inscriptions avec SAG,date
0,CHU Sainte-Justine,31,16,5,0,0,7,186,2,2021-03-09
1,Centre hospitalier de l'Université de Montréal,114,51,58,21,7,48,165,0,2021-03-09
2,Hôpital Louis-H. Lafontaine,42,12,5,3,1,0,13,0,2021-03-09
3,Hôpital Santa Cabrini,129,38,49,17,11,36,91,0,2021-03-09
4,Pavillon Maisonneuve/Pavillon Marcel-Lamoureux,131,54,71,21,4,41,152,0,2021-03-09


In [11]:


print('new data: ', len(new_data), ' rows, ending ', new_data.date.max())

concat_data = pd.concat([old_data, new_data], ignore_index=False)
concat_data = concat_data.drop_duplicates().reset_index(drop=True)
print('concat data: ', len(concat_data), ' rows')

concat_data.to_csv('../dailyMontrealEdStats.csv', index=False)

old_jgh_data = pd.read_csv('../jghDailyVisits.csv')
old_jgh_data['ds'] = pd.to_datetime(old_jgh_data['ds'])
print('old jgh data: ', len(old_jgh_data),
      ' rows, ending ', old_jgh_data.ds.max())

new_jgh_data = df[df.Installation == "L'Hôpital général juif Sir Mortimer B. Davis"][[
    'Nombre inscriptions', 'date']]

new_jgh_data = new_jgh_data.rename(
    columns={'Nombre inscriptions': "y", "date": "ds"})

concat_jgh_data = old_jgh_data.append(
    new_jgh_data, ignore_index=True, sort=True)

print('concat jgh data: ', len(concat_jgh_data))

concat_jgh_data.to_csv('../jghDailyVisits.csv', index=False)

new data:  21  rows, ending  2021-03-09 00:00:00
concat data:  6314  rows
old jgh data:  3231  rows, ending  2021-03-08 00:00:00
concat jgh data:  3232
